In [10]:
!nvidia-smi

Fri May 13 09:59:38 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-PCI...  On   | 00000000:00:06.0 Off |                  Off |
| N/A   34C    P0    33W / 250W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A100-PCI...  On   | 00000000:00:07.0 Off |                  Off |
| N/A   

In [39]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [4]:
import torch
torch.cuda.is_available()

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [1]:
import os
import sys
os.system('ps aux | grep ray > aux_result.txt')

with open('aux_result.txt', 'r') as f:
    result = f.readlines()

print(len(result))

320


In [9]:
results = []
for r in result:
    r = r.split()[1]
    ret = os.system('kill -9 {}'.format(r))
    print('ret: {}'.format(ret))

ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0
ret: 0

sh: 1: kill: No such process

sh: 1: kill: No such process



In [3]:
!nvcc -list-gpu-code

nvcc fatal   : No input files specified; use option --help for more information


In [42]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print('Device:', device)
print('Current cuda device:', torch.cuda.current_device())
print('Count of using GPUs:', torch.cuda.device_count())

Device: cuda
Current cuda device: 0
Count of using GPUs: 1


In [3]:
import os
data_root_path = '/AppData/scannet/'
train_path = data_root_path + 'scans/'
test_path = data_root_path + 'scans_test/'

In [4]:
print('root path:' + data_root_path)
print('train path:' + train_path)
print('test path:' + test_path)

root path:/AppData/scannet/
train path:/AppData/scannet/scans/
test path:/AppData/scannet/scans_test/


In [7]:
import sys
from SensorData import SensorData

In [8]:
paths = [train_path, test_path]
print(paths)

['/AppData/scannet/scans/', '/AppData/scannet/scans_test/']


In [13]:
color_lacks = []
depth_lacks = []
intrinsic_lacks = []
pose_lacks = []

for path in paths:
    dirs = os.listdir(path)
    for dir in dirs:
        dir = path + dir
        if not os.path.exists(os.path.join(dir, 'color')):
            color_lacks.append(dir)
        if not os.path.exists(os.path.join(dir, 'depth')):
            depth_lacks.append(dir)
        if not os.path.exists(os.path.join(dir, 'intrinsic')):
            intrinsic_lacks.append(dir)
        if not os.path.exists(os.path.join(dir, 'pose')):
            pose_lacks.append(dir)

print('color lacks: {}'.format(len(color_lacks)))
print('depth lacks: {}'.format(len(depth_lacks)))
print('intrinsic lacks: {}'.format(len(intrinsic_lacks)))
print('pose lacks: {}'.format(len(pose_lacks)))

color lacks: 0
depth lacks: 0
intrinsic lacks: 0
pose lacks: 0


In [12]:
print(intrinsic_lacks)

['/AppData/scannet/scans/scene0051_01']


In [6]:
save_name = 'all_tsdf_9'
save_path = os.path.join(data_root_path, save_name)
print(data_root_path)
print(save_path)

/AppData/scannet/
/AppData/scannet/all_tsdf_9


In [14]:
import pickle

all_scenes = sorted(os.listdir(save_path))
print('number of scenes: {}'.format(len(all_scenes)))
print('5: {}'.format(all_scenes[3:3+5]))
test = True
# todo: fix for both train/val/test
if not test:
    splits = ['train', 'val']
else:
    splits = ['test']
if not os.path.isdir(os.path.join(data_root_path, 'splits')):
    os.mkdir(os.path.join(data_root_path, 'splits'))    
for split in splits:
    fragments = []
    with open(os.path.join(data_root_path, 'splits', 'scannetv1_{}.txt'.format(split))) as f:
        split_files = f.readlines()
    for scene in all_scenes:
        if 'scene' not in scene:
            continue
        if scene + '\n' in split_files:
            with open(os.path.join(save_path, scene, 'fragments.pkl'), 'rb') as f:
                frag_scene = pickle.load(f)
            fragments.extend(frag_scene)

    with open(os.path.join(save_path, 'fragments_{}.pkl'.format(split)), 'wb') as f:
        pickle.dump(fragments, f)
        print('{} size: {}'.format(split, len(fragments)))

number of scenes: 307
5: ['scene0000_01', 'scene0000_02', 'scene0001_00', 'scene0001_01', 'scene0002_00']
test size: 795
